# Install Bark

In [3]:
import subprocess
import sys
import torch

# 1. Verify GPU is actually active in the session
if torch.cuda.is_available():
    print(f"✅ Kaggle GPU Detected: {torch.cuda.get_device_name(0)}")
else:
    print("❌ GPU NOT DETECTED. Please go to Settings -> Accelerator and select GPU P100.")

# 2. Install Bark without overwriting Kaggle's GPU-enabled PyTorch
print("Installing Bark (GPU-safe mode)...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", 
                       "git+https://github.com/suno-ai/bark.git", "--no-deps"])

# 3. Install the few missing dependencies Bark actually needs (that aren't Torch)
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", 
                       "boto3", "encodec", "funcy", "tqdm"])

print("✅ Installation Complete!")

✅ Kaggle GPU Detected: Tesla P100-PCIE-16GB
Installing Bark (GPU-safe mode)...
✅ Installation Complete!


# Generate an audio with Bark

In [4]:
import torch
import os

# 1. Verification: Force check if CUDA (GPU) is actually reachable
if torch.cuda.is_available():
    device = "cuda"
    print(f"✅ GPU found: {torch.cuda.get_device_name(0)}")
else:
    device = "cpu"
    print("❌ GPU not found. Falling back to CPU. Check your Kaggle settings!")

# --- PATTERN: Fix PyTorch loading (User requirement) ---
_original_load = torch.load
def _patched_load(f, map_location=None, pickle_module=None, *, weights_only=None, mmap=None, **pickle_load_args):
    # We ensure map_location is set to 'cuda' if available
    target_device = 'cuda' if torch.cuda.is_available() else 'cpu'
    return _original_load(f, map_location=target_device, pickle_module=pickle_module, 
                          weights_only=False, mmap=mmap, **pickle_load_args)
torch.load = _patched_load

# --- BARK GENERATION ---
from bark import SAMPLE_RATE, generate_audio, preload_models
from scipy.io.wavfile import write as write_wav

# Choose speaker
SPEAKER = "v2/en_speaker_6"

# YOUR LYRICS
LYRICS = """
[music]
♪ I WAS LOOKING... AT ALL THE LIIIFE ♪
♪ There were plants and birds... and rocks and things ♪
♪ There was sand and hills... and RINGS ♪
♪ The first thing I met... was a fly with a BUZZ ♪
♪ And the sky... with no CLOUDS ♪
[music]
"""

# IMPORTANT: Force GPU in preload_models
print("Loading Bark models onto GPU...")
preload_models(
    text_use_gpu=True,
    coarse_use_gpu=True,
    fine_use_gpu=True,
    codec_use_gpu=True
)

print("Generating voice (GPU accelerated)...")
audio = generate_audio(LYRICS, history_prompt=SPEAKER)

# Save
write_wav("voice_output.wav", SAMPLE_RATE, audio)
print(f"✓ voice_output.wav saved ({len(audio)/SAMPLE_RATE:.1f}s)")

✅ GPU found: Tesla P100-PCIE-16GB
Loading Bark models onto GPU...


text_2.pt:   0%|          | 0.00/5.35G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /root/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th


100%|██████████| 88.9M/88.9M [00:00<00:00, 97.3MB/s]
/usr/local/lib/python3.12/dist-packages/bark/generation.py:175: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with InferenceContext(), torch.inference_mode(), torch.no_grad(), autocast():


Generating voice (GPU accelerated)...


100%|██████████| 36/36 [00:36<00:00,  1.02s/it]


✓ voice_output.wav saved (14.4s)


# Play the audio 

In [5]:
from IPython.display import Audio, display

audio_path = "/kaggle/working/voice_output.wav"
display(Audio(filename=audio_path))
